In [1]:
#ee version 0.1.221
##geemap version 0.6.13

In [2]:
import ee
import geemap
import os
import numpy as np
from matplotlib import pyplot as plt
import matplotlib

ModuleNotFoundError: No module named 'ee'

In [ ]:
print(ee.__version__)
print(geemap.__version__)
ee.Initialize()

In [ ]:
manaus_shp = '/home/jbk/projects/climate/tropics/venus/footprints/venus_fgmanaus_footprint/venus_fgmanaus_footprint.shp'
manaus = geemap.shp_to_ee(manaus_shp)
info = manaus.geometry()
manaus = ee.Geometry(info)

In [ ]:
atto_shp = '/home/jbk/projects/climate/tropics/venus/footprints/venus_atto_footprint/venus_atto_footprint.shp'
atto = geemap.shp_to_ee(atto_shp)
info = atto.geometry()
atto = ee.Geometry(info)

In [ ]:
corumba_shp = '/home/jbk/projects/climate/tropics/venus/footprints/venus_corumba_footprint/venus_corumba_footprint.shp'
corumba = geemap.shp_to_ee(corumba_shp)
info = corumba.geometry()
corumba = ee.Geometry(info)

In [ ]:
mato_shp = '/home/jbk/projects/climate/tropics/venus/footprints/venus_mato_footprint/venus_mato_footprint.shp'
mato = geemap.shp_to_ee(mato_shp)
info = mato.geometry()
mato = ee.Geometry(info)

In [ ]:
saop_shp = '/home/jbk/projects/climate/tropics/venus/footprints/venus_saop_footprint/venus_saop_footprint.shp'
saop = geemap.shp_to_ee(saop_shp)
info = saop.geometry()
saop = ee.Geometry(info)

In [ ]:
#Select geom of interest to export
fc = manaus

In [ ]:
#Calculate NDRE and set palette
def addNDRE(image):
    ndre = image.normalizedDifference(['B8', 'B5']).rename('NDRE')
    return image.addBands(ndre)

In [ ]:
#Create cloud mask
cloudBitMask = ee.Number(2).pow(10).int() #bit 10 is clouds
cirrusBitMask = ee.Number(2).pow(11).int() #bit 11 is cirrus clouds
def maskS2clouds(image):
    qa = image.select('QA60')
    # Both flags should be set to zero, indicating clear conditions.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1<< 11
    mask = (qa.bitwiseAnd(cloudBitMask).eq(0).And(
             qa.bitwiseAnd(cirrusBitMask).eq(0)))
    return image.updateMask(mask)

In [ ]:
#Load sentinel-2 data
s2 = (ee.ImageCollection('COPERNICUS/S2_SR')#Select S2 1-C level data
      .select(['B5', 'B8', 'QA60']) #Select necessary bands to reduce memory usage
      .filterBounds(fc) #Narrow data spatially
      .filterDate(ee.Date('2017-01-01'), ee.Date('2019-12-31')) #Select years of interest
      .sort('CLOUDY_PIXEL_PERCENTAGE', False) #Put smallest %cloud image on top of stack
      .map(addNDRE)) #calculate NDRE and add as band

In [ ]:
#Create collection with cloud mask
col = s2.map(maskS2clouds)

In [ ]:
#15-day step
years = ee.List.sequence(2017, 2019)
step = ee.List.sequence(1, 365, 15)

def byYear(y):
    y = ee.Number(y)
    def byStep(d):
        d = ee.Number(d)
        return (col
                .filter(ee.Filter.calendarRange(y, y.add(1), 'year')) #yearly step
                .filter(ee.Filter.calendarRange(d, d.add(15), 'day_of_year')) #15-day step
                .qualityMosaic('NDRE') #Max greenness
                .set('step', [d, y])) #Add properties
    return step.map(byStep)

col = ee.ImageCollection.fromImages(years.map(byYear).flatten())
col = col.map(lambda image: image.clip(fc)).select('NDRE')

In [ ]:
##ONLY RUN THIS BLOCK FOR ATTO, OTHERWISE SKIP##
#Atto removal: 5, 24
col = (col.filter(ee.Filter.neq('step', [76, 2016])) #5
       .filter(ee.Filter.neq('step', [361, 2016]))) #24

In [ ]:
##ONLY RUN THIS BLOCK FOR SAOP, OTHERWISE SKIP##
#Saop removal: 9, 24, 99
col = (col.filter(ee.Filter.neq('step', [361, 2019])) #99
       .filter(ee.Filter.neq('step', [361, 2016])) #24
       .filter(ee.Filter.neq('step', [136, 2016]))) #9

In [ ]:
#Export images from a collection
out_dir = '/home/jbk/projects/climate/tropics/sentinel-2/15day_composites/manaus/L2A/'
geemap.ee_export_image_collection(col, out_dir, region = fc, scale = 20, file_per_band=False)

## Monthly composites

In [ ]:
mcol = s2.map(maskS2clouds)

#Monthly step
#https://gis.stackexchange.com/questions/301165/how-to-get-monthly-averages-from-earth-engine-in-the-python-api
months = ee.List.sequence(1, 12) #Create a list with nums 1-12
years = ee.List.sequence(2016, 2016)

def byYear(y):
    def byMonth(m):
        return (mcol
                .filter(ee.Filter.calendarRange(y, y, 'year'))
                .filter(ee.Filter.calendarRange(m, m, 'month'))
                .qualityMosaic('NDRE') #What is best way to compile images before running stats?
                .set('month', m).set('year', y))
    return months.map(byMonth)

mcol = ee.ImageCollection.fromImages(years.map(byYear).flatten())
mcol = mcol.map(lambda image: image.clip(fc)).select('NDRE') #Choose JUST NDRE band and clip to region

In [ ]:
#Export images from a collection
out_dir = '/home/jbk/projects/climate/tropics/sentinel-2/monthly_composites/manaus/qm/'
geemap.ee_export_image_collection(mcol, out_dir, region = fc, scale = 20, file_per_band=False)

## Visualize Map

In [ ]:
Map = geemap.Map(center = (-2.609097222, -60.20929722), zoom = 10)
brma2_point = ee.Geometry.Point(-60.20929722, -2.609097222)
brma2_region = ee.Geometry.Rectangle(-61.00, -3.0, -60.00, -2.0)
Map
#Map.addLayer(brma2_region, {}, 'Manauas Region')

In [ ]:
#Load sentinel-2 data
s2 = (ee.ImageCollection('COPERNICUS/S2')#Select S2 1-C level data
#      .select(['B5', 'B8', 'QA60']) #Select necessary bands to reduce memory usage
#      .filterBounds(brma2_region) #Narrow data spatially
      .filterDate(ee.Date('2019-04-01'), ee.Date('2019-05-30')) #Select years of interest
      .sort('CLOUDY_PIXEL_PERCENTAGE', False) #Put smallest %cloud image on top of stack
      .map(addNDRE)) #calculate NDRE and add as band

In [ ]:
maxval = s2.max()

visParams = {
  'bands': ['B4', 'B3', 'B2'],
  'min': 0,
  'max': 3000,
}

Map.addLayer(maxval, visParams, 'MaxVal')
#Map.addLayer(s2, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}, 'RGB Manaus Clouds')

In [ ]:
maxndre = s2.qualityMosaic('NDRE')
print(maxndre.getInfo())

In [ ]:
#maxndre = s2.median()
palette = ['#d73027', '#f46d43', '#fdae61', '#fee08b', '#d9ef8b', '#a6d96a', '#66bd63', '#1a9850']
Map.addLayer(maxndre.select('NDRE'), {'palette': palette}, 'NDRE')

In [ ]:
#Export images from a collection
out_dir = '/home/jbk/projects/climate/tropics/sentinel-2/monthly_composites/manaus/qm/'
geemap.ee_export_image_collection(mcol, out_dir, region = fc, scale = 20, file_per_band=False)

In [ ]:
toa = (ee.ImageCollection('COPERNICUS/S2')
  .filterBounds(brma2_region)
  .filterDate(ee.Date('2019-01-01'), ee.Date('2019-02-01')))

sr = (ee.ImageCollection('COPERNICUS/S2_SR')
  .filterBounds(brma2_region)
  .filterDate(ee.Date('2019-01-01'), ee.Date('2019-06-01')))

def rescale(image): 
    rescaledImage = image.multiply(0.0001);
    return rescaledImage.copyProperties(image, image.propertyNames());


toa = toa.map(rescale)
sr = sr.map(rescale)

#Map.addLayer(toa.max(), {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 1}, 'top of atmosphere')
Map.addLayer(sr.max(), {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 1}, 'surface reflectance')


In [ ]:
toa.getInfo()